In [17]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from datetime import date, timedelta
import time

In [3]:
# Set all the places that the bot is going to extract data from.
cities = '''Ailsa Craig
Alfred
Ameliasburgh
Arkona
Azilda
Barrys Bay
Blackstock
Blenheim
Bobcaygeon
Bothwell
Brechin
Brights Grove
Caledon East
Caledon Village
Camlachie
Cayuga
Cedar Valley
Cheltenham
Chesley
Claremont
Clarksburg
Cobden
Colborne
Coldwater
Corunna
Crysler
Delhi
Dundalk
Dunrobin
Dutton
East Garafraxa
Elliot Lake
Elmvale
Emeryville
Erin
Gananoque
Garson
Goderich
Goodwood
Grand Bend
Grand Valley
Greater Napanee
Grey Highlands
Hagersville
Halton Hills
Hampton
Hanover
Hawkesbury
Hawkestone
Hepworth
Ilderton
Ingleside
Jordan
Kemble
Kettleby
Kincardine
Kirkfield
Kleinburg Station
Lambton Shores
Limehouse
Lions Head
Listowel
Little Britain
Loretto
Lowbanks
Lucan
Madoc
Mallorytown
Markdale
Marmora
Merrickville
Midhurst
Millbrook
Millgrove
Minesing
Mount Brydges
Mount Forest
New Hamburg
Norfolk
North Bay
North Gower
Nottawa
Odessa
Oro
Oro Station
Pakenham
Paris
Parry Sound
Pefferlaw
Pembroke
Petawawa
Picton
Plantagenet
Pointe Aux Roches
Point Edward
Port Dover
Port Elgin
Port Severn
Prescott
Prince Edward
Puslinch
Queenston
Renfrew
Ridgeville
Rockwood
Rosemont
Saint Marys
Sauble Beach
Schreiber
Severn
Singhampton
Smooth Rock Falls
Southampton
Southwold
Springwater
Sunderland
Sutton
Tara
Terra Cotta
Vars
Wahnapitae
Wainfleet
Washago
Waterford
Wellesley
Winchester
Woodlawn
Wyoming'''

# Transform the string in a way that could be used in the url and split it on a list.
cities = cities.lower().replace(' ','-')
cities = cities.split('\n')

In [4]:
# Download the lastest version of the Chrome Driver.
service = Service(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.58M/6.58M [00:00<00:00, 10.5MB/s]


# Selenium Web Scraping Steps

1. Initialize the browser with implicitly wait of 30 seconds
2. Access the site https://rentals.ca/
3. Add the name of the city in the url
4. Open the browser with maximized window
5. Check the number of total results
6. Check Pagination
7. 

In [35]:
# Step 1 - Initialize the browser with implicitly wait of 30 seconds.
browser = webdriver.Chrome(service=service)
browser.implicitly_wait(30)

# Step 2 and 3 - Access the site https://rentals.ca/ with the city name.
city = 'arkona'
browser.get(f'https://rentals.ca/{city}')

# Step 4 - Open the browser with maximized window in order to the bot be able to collect rent information.
browser.maximize_window()

# Step 5 - CLick on the property
browser.find_element(By.CLASS_NAME, 'listing-card__details-link').click()
time.sleep(5)

# Step 6 - Get all html content from the page to parse with bs4
page_content = browser.page_source
rent_information = BeautifulSoup(page_content, 'html.parser')

# Step 7 - Define all the desire variables

# id = rent_information.find('data-property-id')

# Top Panel
updated = rent_information.find('p', attrs={'class':'listing-card__last-updated'}).text
adress = rent_information.find('h2', attrs={'class':'listing-card__title'}).text

main_features = rent_information.find('ul', attrs={'class':'listing-card__main-features'})
main_features = [x.get_text() for x in main_features.find_all('span')]
pets = 1 if 'Pets' in main_features else 0

# Floor Plans
floor_plans = rent_information.find('div', attrs={'class':'floor-plan-group'})

Nbed = floor_plans.find('span').text
price = floor_plans.find('li', attrs={'class':'unit-details__infos--price'}).text
Nbath = floor_plans.find('li', attrs={'class':'unit-details__infos--baths'}).text
size = floor_plans.find('li', attrs={'class':'unit-details__infos--dimensions'}).text

# Utilities Included
utilities_panel = rent_information.find('ul', attrs={'class':'listing-utilities'})
utilities = [x.get_text() for x in utilities_panel.find_all('li')]

# About Panel
about_highlight = rent_information.find('ul', attrs={'class':'listing-highlighted-info'})

property_type = about_highlight.find('h4', attrs={'data-msgid':'Property Type'}).find_next('p').text
property_subtype = about_highlight.find('h4', attrs={'data-msgid':'Property Sub-type'}).find_next('p').text
parking_type = about_highlight.find('h4', attrs={'data-msgid':'Parking Type'}).find_next('p').text
parking_spots = about_highlight.find('h4', attrs={'data-msgid':'Parking Spots'}).find_next('p').text
lease_term = about_highlight.find('h4', attrs={'data-msgid':'Lease Term'}).find_next('p').text
short_term = 1 if about_highlight.find('h4', attrs={'data-msgid':'Short-term'}).find_next('p').text == 'Yes' else 0
furnished = 1 if about_highlight.find('h4', attrs={'data-msgid':'Furnished'}).find_next('p').text == 'Yes' else 0
year_built = about_highlight.find('h4', attrs={'data-msgid':'Year Built'}).find_next('p').text

# Features & Amenities
browser.find_element(By.CLASS_NAME, 'menu-panel-slider__list').click()

features_amenities_panel_type = rent_information.find('h4', attrs={'class':'menu-panel-slider__panel-heading'}).text

features_amenities_panel = rent_information.find('ul', attrs={'class':'listing-features-and-amenities__content'})
features_amenities = [x.get_text() for x in features_amenities_panel.find_all('li')]

browser.find_element(By.CLASS_NAME, 'menu-panel-slider__panel-close').click()

# Neighbourhood Score

NS_daycares = rent_information.find('li', attrs={'data-id':'daycares'}).find_all('span')[-1].text
NS_quiet = rent_information.find('li', attrs={'data-id':'quiet'}).find_all('span')[-1].text
NS_high_schools = rent_information.find('li', attrs={'data-id':'high_schools'}).find_all('span')[-1].text
NS_pedestrian_friendly = rent_information.find('li', attrs={'data-id':'pedestrian_friendly'}).find_all('span')[-1].text
NS_groceries_stores = rent_information.find('li', attrs={'data-id':'groceries'}).find_all('span')[-1].text
NS_elementary_schools = rent_information.find('li', attrs={'data-id':'primary_schools'}).find_all('span')[-1].text
NS_car_friendly = rent_information.find('li', attrs={'data-id':'car_friendly'}).find_all('span')[-1].text
NS_coffee_shops = rent_information.find('li', attrs={'data-id':'cafes'}).find_all('span')[-1].text
NS_restaurants = rent_information.find('li', attrs={'data-id':'restaurants'}).find_all('span')[-1].text
NS_shopping = rent_information.find('li', attrs={'data-id':'shopping'}).find_all('span')[-1].text
NS_vibrancy = rent_information.find('li', attrs={'data-id':'vibrant'}).find_all('span')[-1].text
NS_nightlife = rent_information.find('li', attrs={'data-id':'nightlife'}).find_all('span')[-1].text

# Summary

keywords = rent_information.find('p', attrs={'class':'listing-summary__tags'}).find('em').text


# Data Property ID**
# Realtor**
# Availability**
# Photos**


In [34]:
print(about_highlight.prettify())

<ul class="listing-highlighted-info">
 <li class="listing-highlighted-info__item">
  <h4 data-current-language="en" data-msgid="Property Type">
   Property Type
  </h4>
  <p>
   Apartment
  </p>
 </li>
 <li class="listing-highlighted-info__item">
  <h4 data-current-language="en" data-msgid="Property Sub-type">
   Property Sub-type
  </h4>
  <p>
   Apartment
  </p>
 </li>
 <li class="listing-highlighted-info__item">
  <h4 data-current-language="en" data-msgid="Parking Type">
   Parking Type
  </h4>
  <p>
   Driveway
  </p>
 </li>
 <li class="listing-highlighted-info__item">
  <h4 data-current-language="en" data-msgid="Parking Spots">
   Parking Spots
  </h4>
  <p>
   1
  </p>
 </li>
 <li class="listing-highlighted-info__item">
  <h4 data-current-language="en" data-msgid="Lease Term">
   Lease Term
  </h4>
  <p>
   1-year
  </p>
 </li>
 <li class="listing-highlighted-info__item">
  <h4 data-current-language="en" data-msgid="Short-term">
   Short-term
  </h4>
  <p>
   No
  </p>
 </li>
 <l

In [ ]:
print(rent_information.prettify())